# Time-Series-Library 项目多通道预测机制解析

你好！这个 Notebook 将为你详细解释当前项目中关于多通道（多变量）时间序列预测的一些核心问题：

1.  **输入与输出格式**：在多通道预测中，数据是如何被组织并送入模型的？模型的输出又是什么样的？
2.  **模型与数据的协同工作**：从原始数据到模型预测，整个流程是怎样的？
3.  **MAE 和 MSE 的计算**：对于多通道的预测结果，我们如何评估其准确性？

让我们开始吧！

## 1. 输入与输出格式

在时间序列预测中，我们通常处理的是三维张量（Tensor），其维度结构为 `(batch_size, sequence_length, num_features)`。

-   `batch_size`: 批处理大小，表示一次送入模型的样本数量。
-   `sequence_length`: 序列长度，表示每个样本包含的时间步长。
-   `num_features`: 特征数量，即通道数。在单变量预测中，`num_features`为1；在多变量预测中，`num_features`大于1。

---

### 输入数据 (`batch_x`)

-   **格式**: `(batch_size, seq_len, num_features)`
-   **说明**: 这是模型的编码器（Encoder）输入。`seq_len` 是我们提供给模型的历史数据长度（由参数 `seq_len` 控制）。`num_features` 是数据集中包含的变量数量（例如，对于 ETT 数据集，有7个变量）。

### 标签数据 (`batch_y`)

-   **格式**: `(batch_size, label_len + pred_len, num_features)`
-   **说明**: 这部分数据比较特殊，它包含了**两部分**：
    1.  `label_len` 部分：这是提供给解码器（Decoder）作为初始输入的一部分历史信息。它通常是输入序列 `batch_x` 的后半部分。
    2.  `pred_len` 部分：这是模型需要预测的目标序列。在训练时，这部分是真实的未来数据，用于计算损失；在预测时，这部分是未知的。

在代码 `exp/exp_long_term_forecasting.py` 的训练循环中，解码器的输入 `dec_inp` 是这样构建的：

```python
# 创建一个形状为 (batch_size, pred_len, num_features) 的零张量
dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
# 将 label_len 部分的真实数据和上述零张量拼接起来
dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float()
```

这是一种经典的 "teacher forcing" 模式的变体：在解码器的开始阶段，我们给它一小段真实数据（`label_len`），然后让它自回归地预测未来 `pred_len` 长度的序列。

### 模型输出 (`outputs`)

-   **格式**: `(batch_size, pred_len, num_features)`
-   **说明**: 模型的最终输出是预测未来 `pred_len` 时间步长的序列。`num_features` 的数量取决于预测任务的类型（由参数 `features` 控制）。

---

### 多通道预测的两种模式 (`M` vs `MS`)

项目通过 `features` 参数支持两种主要的多通道预测模式：

1.  **多变量预测多变量 (M: Multivariate -> Multivariate)**
    -   **输入**: `(batch_size, seq_len, C_in)`，其中 `C_in` 是所有输入特征数。
    -   **输出**: `(batch_size, pred_len, C_out)`，其中 `C_out` 是所有需要预测的特征数。在本项目中，通常 `C_in == C_out`。
    -   **目标**: 模型利用所有变量的历史信息，来预测所有变量的未来值。

2.  **多变量预测单变量 (MS: Multivariate -> Univariate)**
    -   **输入**: `(batch_size, seq_len, C_in)`，输入与 `M` 模式相同。
    -   **输出**: `(batch_size, pred_len, 1)`。
    -   **目标**: 模型利用所有变量的历史信息，但只预测**一个**目标变量的未来值。

在 `exp_long_term_forecasting.py` 中，以下代码片段处理了这两种模式的输出：

```python
# f_dim (feature dimension) 在 MS 模式下为 -1，在 M 模式下为 0
f_dim = -1 if self.args.features == 'MS' else 0
# 从模型输出中选择相应的维度
outputs = outputs[:, -self.args.pred_len:, f_dim:]
batch_y = batch_y[:, -self.args.pred_len:, f_dim:]
```

-   如果 `features` 是 `M`，`f_dim` 为 `0`，`outputs[:, :, 0:]` 表示取所有特征通道，输出形状为 `(batch_size, pred_len, num_features)`。
-   如果 `features` 是 `MS`，`f_dim` 为 `-1`，`outputs[:, :, -1:]` 表示只取最后一个特征通道（默认目标变量），输出形状为 `(batch_size, pred_len, 1)`。


## 2. 模型与数据的协同工作流程

整个流程从 `run.py` 开始，通过命令行参数配置实验，然后由 `Exp_Long_Term_Forecast` 类（在 `exp/exp_long_term_forecasting.py` 中）执行。

以下是数据流转的关键步骤：

1.  **获取数据 (`_get_data`)**:
    -   该方法调用 `data_provider/data_factory.py` 中的 `data_provider` 函数。
    -   `data_provider` 根据参数（如 `data`, `root_path`）选择合适的 `DataLoader`（例如 `Dataset_ETT_hour`）。
    -   `DataLoader` 负责从 `.csv` 文件读取数据，进行标准化（Normalization），并根据 `seq_len`, `label_len`, `pred_len` 将长序列切分成一个个的训练样本 `(batch_x, batch_y)`。

2.  **构建模型 (`_build_model`)**:
    -   根据 `args.model`（例如 `Autoformer`）从 `models` 目录中实例化对应的模型。
    -   模型接收 `args` 作为参数，从而可以根据配置动态调整内部结构（如层数、头数等）。

3.  **训练 (`train` 方法)**:
    -   从 `train_loader` 中获取一批数据 `(batch_x, batch_y, batch_x_mark, batch_y_mark)`。
    -   将数据送入模型 `self.model(...)`，得到预测输出 `outputs`。
    -   根据 `features` 参数对 `outputs` 和 `batch_y` 进行切片，确保比较的维度一致。
    -   使用 `nn.MSELoss()` 作为损失函数 `criterion`，计算 `loss = criterion(outputs, batch_y)`。
    -   反向传播并更新模型权重。

4.  **评估 (`test` 方法)**:
    -   加载训练好的最佳模型。
    -   从 `test_loader` 获取数据，进行预测。
    -   将所有批次的预测结果 `preds` 和真实结果 `trues` 收集起来。
    -   调用 `utils/metrics.py` 中的 `metric` 函数，传入 `preds` 和 `trues` 计算最终的评估指标。

这个流程确保了无论模型结构如何，只要它遵循相同的输入输出约定，就可以与数据处理和训练评估流程无缝集成。


## 3. MAE 和 MSE 的计算方法

在多通道预测（`M` 模式）中，模型的输出和真实值都是三维的 `(num_samples, pred_len, num_features)`。那么，MAE 和 MSE 是如何计算的呢？

让我们看一下 `utils/metrics.py` 中的 `metric` 函数：

```python
import numpy as np

def RSE(pred, true):
    return np.sqrt(np.sum((true - pred) ** 2)) / np.sqrt(np.sum((true - true.mean()) ** 2))

def CORR(pred, true):
    # ... (omitted for brevity) ...

def MAE(pred, true):
    return np.mean(np.abs(pred - true))

def MSE(pred, true):
    return np.mean((pred - true) ** 2)

def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

def MAPE(pred, true):
    # ... (omitted for brevity) ...

def MSPE(pred, true):
    # ... (omitted for brevity) ...

def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)

    return mae, mse, rmse, mape, mspe
```

关键在于 `np.mean()` 的行为。当 `pred` 和 `true` 是多维数组时（例如 `(N, L, C)`），`np.mean(np.abs(pred - true))` 会计算**所有元素**绝对误差的平均值。

**这意味着，在多通道预测中，MAE 和 MSE 是在所有样本、所有未来时间步、所有特征通道上计算的误差的总体平均值。**

-   **MSE**: `mean((pred - true)^2)`
-   **MAE**: `mean(|pred - true|)`

这种计算方法给出了一个单一的、总体的性能指标，衡量模型在整个预测任务上的平均表现。它不会区分哪个通道预测得更好或更差，而是将所有通道的误差同等对待。

---

### 总结

-   **输入输出**：使用 `(batch, seq_len, features)` 格式的张量，并通过 `features` 参数 (`M` 或 `MS`) 灵活支持多对多和多对一的预测。
-   **工作流程**：通过标准化的数据提供器、模型接口和训练循环，实现了高度模块化的实验流程。
-   **指标计算**：MAE 和 MSE 通过对所有维度（样本、时间步、特征）的误差求平均，提供了一个综合的性能评估。

希望这个 Notebook 能帮助你更好地理解该项目的代码！